# Install and Import Libraries

In [3]:
!pip install datasets transformers[sentencepiece] sacrebleu -q tokenizers evaluate rouge_score torch huggingface_hub -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 72.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.1 MB/s eta 0:00:00


In [4]:
import os
import sys
import transformers
import tensorflow as tf
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from transformers import AdamWeightDecay
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM

2025-05-02 08:00:19.281906: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746172819.486177      40 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746172819.542679      40 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Load Model and Dataset

In [5]:
model_checkpoint = "Helsinki-NLP/opus-mt-en-de"

In [6]:
datasets = load_dataset("Darth-Vaderr/English-German")

README.md:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

FT_Dataset.jsonl:   0%|          | 0.00/13.3G [00:00<?, ?B/s]

Final_Dataset_v3.jsonl:   0%|          | 0.00/13.3G [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Loading dataset shards:   0%|          | 0/47 [00:00<?, ?it/s]

In [7]:
datasets

DatasetDict({
    train: Dataset({
        features: ['German', 'English'],
        num_rows: 97329967
    })
})

In [8]:
datasets['train'][1]

{'German': 'Ich erkläre die am Freitag, dem 15. Dezember 2000, unterbrochene Sitzungsperiode des Europäischen Parlaments für wieder aufgenommen.',
 'English': 'I declare resumed the session of the European Parliament adjourned on Friday, 15 December 2000.'}

In [9]:
from datasets import DatasetDict
shuffled = datasets['train'].shuffle(seed=42)

# Step 2: Split into train (15000), validation (1000), test (1000)
train_sample = shuffled.select(range(15000))
val_sample = shuffled.select(range(15000, 16000))
test_sample = shuffled.select(range(16000, 17000))

# Step 3: Create a new DatasetDict with the samples
small_dataset = DatasetDict({
    'train': train_sample,
    'validation': val_sample,
    'test': test_sample
})


In [10]:
small_dataset

DatasetDict({
    train: Dataset({
        features: ['German', 'English'],
        num_rows: 15000
    })
    validation: Dataset({
        features: ['German', 'English'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['German', 'English'],
        num_rows: 1000
    })
})

In [11]:
small_dataset['train'][1]

{'German': 'Da noch nicht jeder das Konzept kennt, muss den Teilnehmern in einer Einleitung das Prozedere und dessen Eigenheiten erläutert werden.',
 'English': 'Since not everyone is familiar with this concept, the procedure and its peculiarities must be explained to the participants.'}

In [12]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/768k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/797k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [13]:
tokenizer("Hello, this is a sentence!")

{'input_ids': [16816, 2, 60, 19, 14, 12512, 68, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [14]:
with tokenizer.as_target_tokenizer():
    print(tokenizer(["Ich komme aus banha"]))

{'input_ids': [[105, 9158, 74, 17, 6848, 1803, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1]]}


/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


# Preprocessing

In [15]:
max_input_length = 128
max_target_length = 128

source_lang = "English"
target_lang = "German"


def preprocess_function(examples):
    inputs = [sentence for sentence in examples[source_lang]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Tokenize the targets (highlights)
    labels = tokenizer(
        text_target=examples[target_lang],
        max_length=max_target_length,
        truncation=True,
    )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [16]:

# Apply preprocessing to each split
tokenized_dataset = {
    split: ds.map(preprocess_function, batched=True)
    for split, ds in small_dataset.items()
}


Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

# Training

In [17]:
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

tf_model.h5:   0%|          | 0.00/298M [00:00<?, ?B/s]

I0000 00:00:1746173338.547583      40 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1746173338.548252      40 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5
All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at Helsinki-NLP/opus-mt-en-de.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [18]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

In [19]:
generation_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf", pad_to_multiple_of=128)

In [20]:
batch_size = 16
learning_rate = 2e-5
weight_decay = 0.01
num_train_epochs = 1

In [21]:
train_dataset = model.prepare_tf_dataset(
    tokenized_dataset["train"],
    batch_size=batch_size,
    shuffle=True,
    collate_fn=data_collator,
)

In [22]:
validation_dataset = model.prepare_tf_dataset(
    tokenized_dataset["validation"],
    batch_size=batch_size,
    shuffle=False,
    collate_fn=data_collator,
)

In [23]:
generation_dataset = model.prepare_tf_dataset(
    tokenized_dataset["test"],
    batch_size=8,
    shuffle=False,
    collate_fn=generation_data_collator,
)

In [24]:
optimizer = AdamWeightDecay(learning_rate=learning_rate, weight_decay_rate=weight_decay)
model.compile(optimizer=optimizer)

In [25]:
model.fit(train_dataset, validation_data=validation_dataset, epochs=5)

Epoch 1/5
937/937 [==============================] - 343s 324ms/step - loss: 1.6781 - val_loss: 1.5920
Epoch 2/5
937/937 [==============================] - 304s 324ms/step - loss: 1.4805 - val_loss: 1.6056
Epoch 3/5
937/937 [==============================] - 302s 322ms/step - loss: 1.3280 - val_loss: 1.6262
Epoch 4/5
937/937 [==============================] - 301s 322ms/step - loss: 1.1950 - val_loss: 1.6540
Epoch 5/5
937/937 [==============================] - 302s 322ms/step - loss: 1.0816 - val_loss: 1.6835


In [31]:
model.save_pretrained("./tf_model")


/usr/local/lib/python3.11/dist-packages/transformers/configuration_utils.py:393: UserWarning: Some non-default generation parameters are set in the model config. These should go into either a) `model.generation_config` (as opposed to `model.config`); OR b) a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model).This warning will become an exception in the future.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[58100]]}
  warnings.warn(


In [36]:
tokenizer.save_pretrained("./tok_model")

('./tok_model/tokenizer_config.json',
 './tok_model/special_tokens_map.json',
 './tok_model/vocab.json',
 './tok_model/source.spm',
 './tok_model/target.spm',
 './tok_model/added_tokens.json')

# Model Testing

In [32]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = TFAutoModelForSeq2SeqLM.from_pretrained("./tf_model")

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at ./tf_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


In [33]:
input_text  = "My name is Ziad. I am twenty two years old"

tokenized = tokenizer([input_text], return_tensors='np')
out = model.generate(**tokenized, max_length=128)
print(out)

tf.Tensor(
[[58100  1701  1282    29  5657  1183     3   105   495   338 29263   431
   3283     0 58100]], shape=(1, 15), dtype=int32)


In [34]:
with tokenizer.as_target_tokenizer():
    print(tokenizer.decode(out[0], skip_special_tokens=True))

Mein Name ist Ziad. Ich bin zweiundzwanzig Jahre alt


/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
